Load training data into pandas dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

print(f'{train.head()}\n{train.info()}')

# Initial EDA # 

## Dealing with NaNs ##

In [ ]:
# examine Nans
print(f'Training NaNs:\n{train.isnull().sum()}\n\nTesting NaNs:\n{test.isnull().sum()}')
print(f'\nThe data contains {train.isnull().sum().sum() + test.isnull().sum().sum()} NaNs')

There are alot of NaNs in this dataset.

In [ ]:
# class imbalance including NaNs

fig1, ax = plt.subplots(2, 2, figsize=(10, 10))
idx = 0

for row in [0, 1]:
    for col in [0, 1]:
        feat = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'][idx]
        train[feat].value_counts(dropna=False).plot(kind='pie', ax = ax[row][col])
        idx += 1

The above features are highly imbalanced.

In [ ]:
# distributions of numerical features
original_numerical_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

fig2, ax = plt.subplots(2, 3, figsize=(10, 10))
train[original_numerical_features].hist(ax=ax)

The numerical features are very skewed (apart from Age)

We need to impute the missing values, the question is what method is best?
One way would be to impute with the most common value for each feature as such:
``` python
# extract columns with NaNs
contains_nans = train.columns[train.isnull().any()]

# fills with most common for all cols. Very basic.
for col in contains_nans:
    temp = train[col].value_counts().index[0]
    train[col] = train[col].fillna(temp)
    test[col] = test[col].fillna(temp)
```
However, we can attempt a slightly more foccused imputation for each feature.

In [ ]:
# extract columns with NaNs
contains_nans = train.columns[train.isnull().any()]

# index removes transported to avod error when filling test data
for col in train.isnull().sum().index[0:-1]:
    # check if contains nans and is a float64
    if train[col].isnull().sum() > 0 and train[col].dtypes == 'float64':
        temp = train[col].median()
    else:
        # fill with most common value
        temp = train[col].value_counts().index[0]
        
    train[col] = train[col].fillna(temp)
    test[col] = test[col].fillna(temp)
    
print(f'Training NaNs:\n{train.isnull().sum()}\n\nTesting NaNs:\n{test.isnull().sum()}')
print(f'\nThe data contains {train.isnull().sum().sum() + test.isnull().sum().sum()} NaNs')

<h1><center> Feature creation <center><h1>

Numerous features can be split into more features. The PassengerId has the format of 'XXXX_XX'. We can split the first 4 digits, and last 2 into seperate features. The cabin format is of "deck / number / side (port P or startboard S). We can seperate these into seperate columns. It is a valid assumption that families travel together, and stay in the same rooms. First and Last name features can be created from the original names feature. The 3 original features have been dropped afterwards.

In [ ]:
# splitting PassengerId feature
train[['PassengerId_0', 'PassengerId_1']] = train['PassengerId'].str.split('_', 1, expand=True)
test[['PassengerId_0', 'PassengerId_1']] = test['PassengerId'].str.split('_', 1, expand=True)

# splitting Cabin feature
train[['Deck', 'Number', 'Side']] = train['Cabin'].str.split('/', 2, expand=True)
test[['Deck', 'Number', 'Side']] = test['Cabin'].str.split('/', 2, expand=True)

# splitting Name feature
train[['First name', 'Family name']] = train['Name'].str.split(' ', 1, expand=True)
test[['First name', 'Family name']] = test['Name'].str.split(' ', 1, expand=True)

# drop old features
train.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)
test.drop(['PassengerId', 'Cabin', 'Name'], axis=1, inplace=True)

# snacks 
train['Food/drink'] = train['RoomService'] + train['FoodCourt']
test['Food/drink'] = test['RoomService'] + test['FoodCourt']

# Entertainment
train['Entertainment'] = train['Spa'] + train['VRDeck']
test['Entertainment'] = test['Spa'] + test['VRDeck']

# Total spent
train['TotalSpent'] = train['RoomService'] + train['FoodCourt'] + train['ShoppingMall'] + train['Spa'] + train['VRDeck']
test['TotalSpent'] = test['RoomService'] + test['FoodCourt'] + test['ShoppingMall'] + test['Spa'] + test['VRDeck']


In [ ]:
# baller: CryoSleep = False and TotalSpent > TotalSpent.median()

def is_baller(df):
    mask = (df['CryoSleep'] == False) & (df['TotalSpent'] > df['TotalSpent'].median())    
    return mask
    
train['Baller'] = is_baller(train)
test['Baller'] = is_baller(test)

In [ ]:
# Examining dangerous decks: more people died than not

danger_decks = []

# is danger deck?
for idx, deck in enumerate(train['Deck'].unique()):
    temp = train[train['Deck'] == deck]
    counts = temp['Transported'].value_counts().sort_index(ascending=False).values # list in form of [True count, False count]
    if counts[1] > counts[0]:
        print(f'{deck} is a dangerous deck. {counts[1]} people died compared to the {counts[0]} that survived')
        # easier to read than using a set
        if deck not in danger_decks:
            danger_decks.append(deck)

# new feature
train['DangerDeck'] = np.where(train['Deck'].isin(danger_decks), 1, 0)
test['DangerDeck'] = np.where(test['Deck'].isin(danger_decks), 1, 0)

In [ ]:
numerical_features = []
categorical_features = []

for col in train.columns:
    if (train[col].dtypes == 'float64') and (col not in numerical_features):
        numerical_features.append(col)
    else:
        if (col != 'Transported') and (col not in categorical_features):
            categorical_features.append(col)

print(f'There are {len(numerical_features)} numerical features and {len(categorical_features)} categorical features')            

I'm too curious, so lets examine some log transformations for features.

In [ ]:
# log transform of age, the minus mins helps with negative numbers
def log_transform(df, col):
    transformed = (df[col] - df[col].min() + 1).transform(np.log)
    return transformed

# for each feature, log transform, calculate corr to target
for idx, feat in enumerate(numerical_features):
    transformed_feat = log_transform(train, feat)
    correlation = transformed_feat.corr(train['Transported'])
    # if higher than threshold, add to feature table
    if abs(correlation) > 0.3:
        name = "Log" + feat
        train[name] = transformed_feat
        test[name] = log_transform(test, feat)
        print(f'{feat} {correlation}')
        
print(f'Training:\n{train.head()}\nTesting:\n{test.head()}')

In [ ]:
train.info()

## Scaling and encoding features ##

Scaling features. We will examine the difference between the MinMax and Standard scaler from the sklearn library.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

min_max_scaler = MinMaxScaler()
stand_scaler = StandardScaler()

train[numerical_features] = min_max_scaler.fit_transform(train[numerical_features])
test[numerical_features] = min_max_scaler.fit_transform(test[numerical_features])

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_df_cols(df, columns):
    for col in columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype('str'))   
    return df

train = encode_df_cols(train, categorical_features)
test = encode_df_cols(test, categorical_features)

print(f'Training:\n{train.head()}\nTesting:\n{test.head()}')

In [ ]:
# double check no NaNs exist
train.isnull().sum()

# Feature Selections #

This sections aims to evaluate each feature in relation to the target ('Transported'). This should assist our models decisions. The numerical and categorical features will be evaluated seperately.

<h2> Numerical Features <h2>

In [ ]:
# update feature lists
for col in train.columns:
    if (train[col].dtypes == 'float64') and (col not in numerical_features):
        numerical_features.append(col)
    else:
        if (col != 'Transported') and (col not in categorical_features):
            categorical_features.append(col)

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(3, 3, figsize=(15, 15))
idx = 0

for row in range(0,3):
    for col in range(0,3):
        sns.kdeplot(data=train, x=numerical_features[idx], hue="Transported", ax=ax[row][col])
        idx+=1

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

# astype(float) to avoid empty dataframe error
numerical_corrs = train[numerical_features + ['Transported']].astype(float).corr()
sns.heatmap(numerical_corrs, annot=True, ax=ax)

In [ ]:
# sort numerical features from largest to smallest abs correlation
print(numerical_corrs['Transported'].abs().sort_values(ascending=False))

## Categorical features ##

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
categorical_corrs = train[categorical_features + ['Transported']].corr()
sns.heatmap(categorical_corrs, annot=True)

In [ ]:
# sort categorical features from largest to smallest abs correlation
print(categorical_corrs['Transported'].abs().sort_values(ascending=False))

## Dropping redundant features ##
# Sometimes less is more #

It is clear from above that multiple features provide almost no tangible input to the models.

In [ ]:
# see total features corrs
total_corrs = pd.concat([numerical_corrs['Transported'], categorical_corrs['Transported']]).abs().sort_values(ascending=False)
print(total_corrs)

In [ ]:
# remove the double transported value, and all rows less than 0.1
corr_threshold = 0.15
total_corrs[total_corrs > corr_threshold][1:]

In [ ]:
train = train[total_corrs[total_corrs > corr_threshold][1:].keys()]
test = test[total_corrs[total_corrs > corr_threshold][2:].keys()]

print(f'Training:\n{train.head()}\nTesting:\n{test.head()}')

Split into train into features and target dfs

In [ ]:
Y_train = train['Transported']
X_train = train.loc[:, train.columns != 'Transported']

# Models #

In [ ]:
from sklearn.neural_network import MLPClassifier

neural_net = MLPClassifier(hidden_layer_sizes=(4000, 200, 100, 20), early_stopping=True, validation_fraction=0.2, verbose=True)
neural_net.fit(X_train, Y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn_params = {'n_neighbors': [3, 5, 7]}
knn = GridSearchCV(KNeighborsClassifier(), knn_params, n_jobs=4, refit=True, verbose=1, return_train_score=True)
knn.fit(X_train, Y_train)
print(knn.best_score_)

In [ ]:
from sklearn.svm import SVC

svc_params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
svc = GridSearchCV(SVC(), svc_params, n_jobs=4, verbose=1)
svc.fit(X_train, Y_train)
print(svc.best_score_)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc_params = {'criterion': ['gini', 'entropy'], 'max_depth': range(1,10)}
clf_dtc = GridSearchCV(DecisionTreeClassifier(), dtc_params, n_jobs=4, verbose=1)
clf_dtc.fit(X_train, Y_train)
dtc = clf_dtc.best_estimator_
print(f'{clf_dtc.best_score_}\n{clf_dtc.best_estimator_}')

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_params = {'n_estimators': [100, 300, 500]}
clf = GridSearchCV(AdaBoostClassifier(base_estimator=clf_dtc.best_estimator_), ada_params, n_jobs=4, verbose=1)
clf.fit(X_train, Y_train)
ada = clf.best_estimator_
print(f'{clf.best_score_}\n{clf.best_estimator_}')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {'n_estimators': [100, 300, 500]}
clf = GridSearchCV(RandomForestClassifier(), rfc_params, n_jobs=4, verbose=1)
clf.fit(X_train, Y_train)
rfc = clf.best_estimator_
print(f'{clf.best_score_}\n{clf.best_estimator_}')

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc_params = {'loss': ['deviance', 'exponential'],
              'n_estimators': [100, 200, 300],
              'max_depth': [1, 3, 5]}
clf = GridSearchCV(GradientBoostingClassifier(), gbc_params, n_jobs=4, verbose=1)
clf.fit(X_train, Y_train)
gbc = clf.best_estimator_
print(f'{clf.best_score_}\n{clf.best_estimator_}')

In [ ]:
def create_submission(model):
    submission['Transported'] = model.predict(test)
    submission.to_csv("/kaggle/working/submission.csv", index=False)
    
create_submission(gbc)